## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/bert-131-last/' + '*.h5')
model_path_list += glob.glob('/kaggle/input/bert-131-best/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_1.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_2.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_3.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_4.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_5.h5
/kaggle/input/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_1_last_epoch.h5
/kaggle/input/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_2_last_epoch.h5
/kaggle/input/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_4_last_epoch.h5
/kaggle/input/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_5_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [7]:
def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output_question, sequence_output_question = bert_layer([input_word_ids_question, input_masks_question, segment_ids_question])
    pooled_output_answer, sequence_output_answer = bert_layer([input_word_ids_answer, input_masks_answer, segment_ids_answer])

    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))
weights = [.13, .12, .08, .11, .12, .13, .1, .08, .13]

for index, model_path in enumerate(model_path_list):
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) * weights[index]

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.950288,0.678359,0.188038,0.523528,0.645704,0.521559,0.682886,0.606975,0.628914,...,0.911138,0.936298,0.578028,0.973678,0.976072,0.830323,0.026255,0.033625,0.789042,0.907471
1,46,0.839074,0.488368,0.009405,0.773438,0.755885,0.938405,0.528172,0.462515,0.100673,...,0.650737,0.932644,0.645957,0.969748,0.982100,0.867468,0.919277,0.132040,0.083236,0.879929
2,70,0.867356,0.677329,0.030616,0.784169,0.852125,0.913815,0.571682,0.475693,0.110049,...,0.856173,0.944347,0.578726,0.967749,0.963713,0.844565,0.034256,0.030658,0.840566,0.907130
3,132,0.864122,0.412301,0.010461,0.732485,0.712817,0.901224,0.511642,0.415884,0.095819,...,0.705288,0.950683,0.684268,0.973190,0.979897,0.902323,0.775731,0.166722,0.637287,0.892017
4,200,0.937785,0.498635,0.041297,0.863189,0.761637,0.903245,0.634614,0.597319,0.096876,...,0.670254,0.917063,0.642017,0.955235,0.960841,0.814525,0.266109,0.167451,0.553298,0.869253


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.881799,0.586945,0.035334,0.730154,0.802589,0.853318,0.564318,0.487226,0.233824,...,0.795502,0.931112,0.653858,0.963347,0.972477,0.864812,0.525494,0.142611,0.494536,0.906156
std,2812.670060,0.051508,0.145276,0.054384,0.103534,0.095329,0.122778,0.057709,0.090276,0.209978,...,0.092458,0.032026,0.048971,0.016834,0.014730,0.045704,0.320273,0.068725,0.286749,0.027287
min,39.000000,0.712532,0.335893,0.003768,0.292087,0.443454,0.259444,0.452235,0.339727,0.013034,...,0.561562,0.744205,0.466853,0.883219,0.864340,0.698706,0.010893,0.011112,0.025393,0.797153
25%,2572.000000,0.846602,0.455833,0.009400,0.671866,0.758809,0.830759,0.519046,0.416962,0.074103,...,0.717299,0.913088,0.622011,0.955941,0.966104,0.838342,0.190521,0.091933,0.241124,0.889791
50%,5093.000000,0.883732,0.571855,0.014468,0.735903,0.812105,0.900272,0.554167,0.459087,0.145073,...,0.792282,0.939028,0.651098,0.967212,0.976692,0.873764,0.596896,0.143230,0.489101,0.910800
75%,7482.000000,0.921331,0.714154,0.031988,0.796317,0.863452,0.928630,0.602875,0.538167,0.362405,...,0.883081,0.953721,0.683081,0.974571,0.982405,0.896279,0.823574,0.189937,0.754422,0.925838
max,9640.000000,0.976364,0.900771,0.410940,0.968045,0.975242,0.970378,0.732604,0.747194,0.861702,...,0.960537,0.984851,0.827960,0.988498,0.992155,0.954529,0.955233,0.372349,0.973761,0.956363
